In [ ]:
import torch

cuda_available = torch.cuda.is_available()
print("CUDA Available:", cuda_available)

if cuda_available:
    print("Device Name:", torch.cuda.get_device_name(0))
    print("CUDA Device Count:", torch.cuda.device_count())
    print("Current CUDA Device:", torch.cuda.current_device())

In [ ]:
import os
import cv2
from PIL import Image
import torch
from torch import nn, optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
from torch.optim.lr_scheduler import StepLR
from torchvision.models import MobileNet_V2_Weights

dataset_dir = "/home/mostafabakr/Desktop/Project X/Test_img/detected_hands"  
video_dataset_dir = "/home/mostafabakr/Desktop/Project X/vid data"  
frame_output_dir = "/home/mostafabakr/Desktop/Project X/Test_img/frames"  
save_image_model_path = "/home/mostafabakr/Desktop/Project X/Final_models/asl_image_model.pth"
save_video_model_path = "/home/mostafabakr/Desktop/Project X/Final_models/asl_video_model.pth"
img_size = 224
batch_size = 16
num_epochs = 50
learning_rate = 0.001
validation_split = 0.2
patience = 5
frame_rate = 5 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


def train_model(dataset_dir, save_model_path):
    if not os.path.exists(dataset_dir):
        raise FileNotFoundError(f"Dataset directory '{dataset_dir}' not found.")
    
    dataset = datasets.ImageFolder(root=dataset_dir, transform=transform)
    val_size = int(len(dataset) * validation_split)
    train_size = len(dataset) - val_size
    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    class_names = dataset.classes
    num_classes = len(class_names)

    weights = MobileNet_V2_Weights.IMAGENET1K_V1
    model = models.mobilenet_v2(weights=weights)
    model.classifier[1] = nn.Linear(model.last_channel, num_classes)
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

    best_val_accuracy = 0.0
    early_stop_counter = 0

    print("Starting training...")
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = correct / total * 100

        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = val_correct / val_total * 100

        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"  Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            early_stop_counter = 0
        else:
            early_stop_counter += 1

        if early_stop_counter >= patience:
            print("Early stopping triggered.")
            break

        scheduler.step()

    torch.save(model.state_dict(), save_model_path)
    print(f"Training complete. Model saved to {save_model_path}")

def extract_frames(video_dataset_dir, output_dir, frame_rate=60):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for label_dir in os.listdir(video_dataset_dir):
        label_path = os.path.join(video_dataset_dir, label_dir)
        output_label_dir = os.path.join(output_dir, label_dir)
        os.makedirs(output_label_dir, exist_ok=True)

        for video_file in os.listdir(label_path):
            video_path = os.path.join(label_path, video_file)
            cap = cv2.VideoCapture(video_path)
            frame_count = 0

            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                if frame_count % frame_rate == 0:
                    frame_filename = os.path.join(
                        output_label_dir, f"{os.path.splitext(video_file)[0]}_frame_{frame_count}.jpg"
                    )
                    cv2.imwrite(frame_filename, frame)
                frame_count += 1

            cap.release()

    print("Frame extraction complete.")



if __name__ == "__main__":
    train_model(dataset_dir, save_image_model_path)

    extract_frames(video_dataset_dir, frame_output_dir)
    train_model(frame_output_dir, save_video_model_path)

Using device: cuda
Starting training...
Epoch 1/50
  Train Loss: 0.2394, Train Accuracy: 93.00%
  Val Loss: 0.0839, Val Accuracy: 97.40%
Epoch 2/50
  Train Loss: 0.1108, Train Accuracy: 96.80%
  Val Loss: 0.1819, Val Accuracy: 94.78%
Epoch 3/50
  Train Loss: 0.0819, Train Accuracy: 97.59%
  Val Loss: 0.0443, Val Accuracy: 98.69%
Epoch 4/50
  Train Loss: 0.0669, Train Accuracy: 98.06%
  Val Loss: 0.1099, Val Accuracy: 97.37%
Epoch 5/50
  Train Loss: 0.0603, Train Accuracy: 98.22%
  Val Loss: 0.0354, Val Accuracy: 99.04%
Epoch 6/50
  Train Loss: 0.0182, Train Accuracy: 99.46%
  Val Loss: 0.0158, Val Accuracy: 99.56%
Epoch 7/50
  Train Loss: 0.0090, Train Accuracy: 99.74%
  Val Loss: 0.0133, Val Accuracy: 99.64%
Epoch 8/50
  Train Loss: 0.0052, Train Accuracy: 99.84%
  Val Loss: 0.0122, Val Accuracy: 99.67%
Epoch 9/50
  Train Loss: 0.0035, Train Accuracy: 99.88%
  Val Loss: 0.0115, Val Accuracy: 99.70%
Epoch 10/50
  Train Loss: 0.0024, Train Accuracy: 99.94%
  Val Loss: 0.0117, Val Accura